In [1]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 2.1.0
Default GPU Device: /device:GPU:0


In [8]:
import pandas as pd

full_data = pd.read_csv('result1.csv', header=None)
data = full_data.drop(full_data.columns[0:2], axis=1)
data = data.drop(data.columns[2], axis=1)

print(data.head(10))


         2   3           5          6           7           8           9   \
0  0.112145   0   45.000000   0.255070  104.872465  104.872465  104.744930   
1  0.301299   0  117.458519   0.981370  117.967833  117.967833  117.477148   
2  0.544129   0   82.000000  34.233333  103.883333  103.883333   86.766667   
3  0.801264   0   97.556111   0.256111   98.428056   98.428056   98.300000   
4  0.853760   0   27.000000   5.614722   81.753750   81.753750   78.946389   
5  0.068051   0   13.000000   3.900000   91.950000   91.950000   90.000000   
6  0.272064   0   33.000000  14.000000  126.000000  126.000000  119.000000   
7  0.382543   0  113.000000  13.000000  125.000000  125.000000  119.000000   
8  0.809805   0   57.000000   2.000000   96.000000   96.000000   95.000000   
9  0.991389   0   36.000000  44.000000   93.000000  100.000000   71.000000   

           10        11  12  13     14        15   16  
0  105.000000  0.000556   0  20    0.1  0.000000  104  
1  118.458519  0.007708  -1  

In [9]:
normalised_data=(data-data.min())/(data.max()-data.min())
print(normalised_data.head(10))

         2    3         5         6         7         8         9         10  \
0  0.110609  0.0  0.194444  0.163613  0.169275  0.149604  0.551289  0.105000   
1  0.300105  0.0  0.596992  0.164238  0.191471  0.168285  0.618301  0.118459   
2  0.543374  0.0  0.400000  0.192833  0.167599  0.148193  0.456667  0.121000   
3  0.800974  0.0  0.486423  0.163614  0.158353  0.140411  0.517368  0.098556   
4  0.853565  0.0  0.094444  0.168222  0.130091  0.116624  0.415507  0.084561   
5  0.066435  0.0  0.016667  0.166748  0.147373  0.131170  0.473684  0.093900   
6  0.270817  0.0  0.127778  0.175433  0.205085  0.179743  0.626316  0.133000   
7  0.381496  0.0  0.572222  0.174573  0.203390  0.178317  0.626316  0.132000   
8  0.809530  0.0  0.261111  0.165114  0.154237  0.136947  0.500000  0.097000   
9  0.991443  0.0  0.144444  0.201232  0.149153  0.142653  0.373684  0.115000   

         11   12        13        14        15        16  
0  0.004620  0.5  0.292308  0.000526  0.000000  0.522222  
1

In [11]:
from sklearn.model_selection import train_test_split
import numpy as np
from numpy import newaxis

validation_split = 0.1
train, validation = train_test_split(normalised_data, test_size=validation_split, shuffle=False)

train_x = train.loc[:, :15]
train_y = train.loc[:, 16]
val_x = validation.loc[:, :15]
val_y = validation.loc[:, 16]

x_train = train_x.to_numpy()
y_train = train_y.to_numpy()
x_val = val_x.to_numpy()
y_val = val_y.to_numpy()

x_train = x_train[:,:, newaxis]
x_val = x_val[:,:, newaxis]

print(x_train.shape)


(344327, 13, 1)


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM#, CuDNNLSTM
#from tensorflow.compat.v1.keras.layers import CuDNNLSTM

print(x_train.shape)

model = Sequential()
model.add(LSTM(10, input_shape=(x_train.shape[1:]), return_sequences=False))
model.add(Dropout(0.15))
model.add(Dense(5, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='relu'))

opt = tf.keras.optimizers.Adam(lr=1.5e-5, decay=1e-6)

model.compile(loss='mean_squared_error', optimizer=opt)

history = model.fit(x_train, y_train, epochs=5, batch_size=256, validation_data=(x_val, y_val))

# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# history = model.fit(x_train, y_train, epochs=10, batch_size=128, callbacks=callbacks_list)

(344327, 13, 1)
Train on 344327 samples, validate on 38259 samples
Epoch 1/5
344327/344327 [==============================] - 11s 31us/sample - loss: 0.2392 - val_loss: 0.2143
Epoch 2/5
344327/344327 [==============================] - 8s 24us/sample - loss: 0.1502 - val_loss: 0.1061
Epoch 3/5
344327/344327 [==============================] - 8s 24us/sample - loss: 0.0521 - val_loss: 0.0236
Epoch 4/5
344327/344327 [==============================] - 8s 24us/sample - loss: 0.0165 - val_loss: 0.0133
Epoch 5/5
344327/344327 [==============================] - 8s 24us/sample - loss: 0.0148 - val_loss: 0.0131


NameError: name 'plt' is not defined

In [14]:
test_loss, test_acc = model.evaluate(x_val,  y_val, verbose=1) 

predictions = model.predict(x_val)
print(test_loss)


#print('Test accuracy:', test_acc)
# for num in range(1,5):
#     print(x_val[num])
#     print(y_val[num])
#     model.predict(x_val[num])

38259/38259 [==============================] - 4s 99us/sample - loss: 0.0131


TypeError: cannot unpack non-iterable numpy.float64 object

In [15]:
for num in range(0, 20):
    print("Actual value = " + str(y_val[num]) + ", Predicted Value = " + str(predictions[num]) + ", Difference = " + str(predictions[num]-y_val[num]))
    

NameError: name 'predictions' is not defined

In [6]:
import numpy as np
from numpy import newaxis
## Test on random new data to see if a similar price is produced

new_data = np.asarray([[0.187140, 1, 108.968014, 1.272979, 107.363510, 107.363510, 106.727021, 108.000000, 0.002708, 0.000000, 20, 0, 0.000000, 108.000000]])

for line_list in new_data:
#      line_list = line.split(',')
     new_row = []
     new_row.append(float(line_list[0]))
     new_row.append(float(line_list[1]))
     for i in range(2, len(line_list) - 2):
          mm_index = i - 2
          normalised_value = (float(line_list[i]) - min_values[mm_index])/(max_values[mm_index] - min_values[mm_index])
          new_row.append(normalised_value)
     new_row.append(float(line_list[len(line_list) - 2]))
     desired_output = (float(line_list[len(line_list) - 1]) - 10)/(190 - 10)
        
#print(new_row)
data = np.asarray(new_row)
data = data[newaxis, :]

print(data)

print(desired_output)

normalised_price = model.predict(np.asarray(data))
print(normalised_price[0][0])
price = (normalised_price * (190 - 10)) +  10

print("Actual price = " + str(new_data[0][-1]) + "; Predicted price = " + str(price[0][0]))


[[0.18714    1.         0.5498223  0.16448841 0.17349747 0.15315765
  0.56172116 0.108      0.0225031  0.5        0.29230769 0.
  0.        ]]
0.5444444444444444


NameError: name 'model' is not defined

In [16]:
model.save('DeepTraderv1_underfit.h5')

In [33]:
new_model = tf.keras.models.load_model('DeepTraderv1.h5')

new_model.evaluate(x_val,  y_val, verbose=1) 

63764/63764 [==============================] - 4s 59us/sample - loss: 1.7157e-05


1.715695628211257e-05

In [34]:
new_model.predict(np.asarray(data))

array([[0.5445725]], dtype=float32)